# 双臂机器人 3D 重建 (基于 AirExo-2)



## 1. 导入依赖库

In [14]:
import os
import sys
import h5py
import json
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation
from PIL import Image
from IPython.display import display
import plotly.graph_objects as go

# # 添加 AirExo-2 到 Python 路径
# AIREXO_PATH = "/path/to/AirExo-2"  # 修改为你的 AirExo-2 路径
# sys.path.insert(0, AIREXO_PATH)

# 导入 AirExo-2 模块
from airexo.helpers.urdf_robot import forward_kinematic_single
from airexo.helpers.constants import ROBOT_PREDEFINED_TRANSFORMATION, O3D_RENDER_TRANSFORMATION
from airexo.helpers.renderer import RobotRenderer
from omegaconf import OmegaConf

print("✓ 所有库导入成功!")

✓ 所有库导入成功!


## 2. 配置路径

In [ ]:
"""
标定文件内容
{
'type': 'robot', 
'camera_serials': ['105422061350', '104122064161', '104122061330'], 
'camera_serials_global': ['105422061350'], 
'camera_serial_inhand_left': '104122064161', 
'camera_serial_inhand_right': '104122061330', 
'intrinsics': {
    '105422061350': array([[912.4466 ,   0.     , 633.4127 ],
       [  0.     , 911.4704 , 364.21265],
       [  0.     ,   0.     ,   1.     ]], dtype=float32), 
    '104122064161': array([[915.71423,   0.     , 638.86804],
       [  0.     , 915.29736, 357.55472],
       [  0.     ,   0.     ,   1.     ]], dtype=float32), 
    '104122061330': array([[909.9401 ,   0.     , 626.91187],
       [  0.     , 909.0405 , 354.72583],
       [  0.     ,   0.     ,   1.     ]], dtype=float32)}, 
'extrinsics': {
    '105422061350': array([[-0.03884323, -0.99906784,  0.01883331,  0.01281536],
       [-0.91665816,  0.02812369, -0.39868146,  0.024272  ],
       [ 0.39778018, -0.03274978, -0.91689605,  0.68936044],
       [ 0.        ,  0.        ,  0.        ,  1.        ]],
      dtype=float32), 
    '104122064161': array([[-0.94874346, -0.25861034, -0.18167697,  0.07677209],
       [-0.18934393,  0.9253561 , -0.32842812, -0.03099394],
       [ 0.25305077, -0.2771946 , -0.9268918 ,  0.38643917],
       [ 0.        ,  0.        ,  0.        ,  1.        ]],
      dtype=float32), 
    '104122061330': array([[ 0.9448085 , -0.29227304,  0.14803174, -0.09153452],
       [-0.2114339 , -0.8890973 , -0.4059578 , -0.0019359 ],
       [ 0.25026512,  0.35225344, -0.9018231 ,  0.39790273],
       [ 0.        ,  0.        ,  0.        ,  1.        ]],
      dtype=float32)}, 
'robot_left': {'tcp_pose': array([ 0.47990388,  0.05316564,  0.39616618,  0.1864741 ,  0.5887938 ,
        0.78571755, -0.0346007 ], dtype=float32), 'joint_pos': array([ 0.44278383, -1.0634778 ,  1.3928185 , -1.5582722 , -1.60449   ,
       -0.49928525,  0.33921626], dtype=float32), 'tcp_vel': array([ 0.00069122,  0.00031965, -0.00019603, -0.00065898,  0.00021872,
        0.00270903], dtype=float32), 'joint_vel': array([ 1.3076103e-03, -1.7766005e-03, -1.9677939e-04, -2.6353818e-04,
        1.8236322e-03,  6.1336366e-05, -1.6573255e-03], dtype=float32), 'force_torque': array([-0.06381213, -1.8813924 ,  1.2751839 , -1.0001243 , -0.07868998,
       -0.30840093], dtype=float32)}, 'robot_right': {'tcp_pose': array([ 0.5281115 , -0.00951796,  0.3995147 ,  0.18649498, -0.56400096,
        0.8018281 ,  0.06476301], dtype=float32), 'joint_pos': array([ 0.38581365, -0.7189137 ,  1.9208367 , -1.4884975 , -1.9956506 ,
       -0.03493906,  0.42234862], dtype=float32), 'tcp_vel': array([-1.3955230e-04,  1.0149263e-04,  2.2846512e-05,  4.6651743e-04,
        3.4199504e-04, -7.8015501e-04], dtype=float32), 'joint_vel': array([-0.0002656 , -0.00160371,  0.00064964,  0.00167814, -0.00441418,
        0.00349331, -0.00212246], dtype=float32), 'force_torque': array([ 4.2057753 ,  6.2080684 ,  4.4443674 ,  1.9914904 , -1.7823514 ,
        0.84697425], dtype=float32)}}
"""



In [15]:
# ===== 数据路径配置 =====
SCENE_PATH = "/data/haoxiang/data/FLIPPING_v3/train/scene_0001"
LOWDIM_H5_PATH = os.path.join(SCENE_PATH, "lowdim/lowdim.h5")
CALIB_PATH = "/data/haoxiang/data/airexo2/task_0013/calib/1737548651048.npy"

# ===== 标定数据 =====
CALIB_RAW = np.load(CALIB_PATH, allow_pickle=True).item()

# ===== 双臂 URDF 配置 =====
URDF_FILE_LEFT = "airexo/urdf_models/robot/left_robot_inhand.urdf"
URDF_FILE_RIGHT = "airexo/urdf_models/robot/right_robot_inhand.urdf"

# ===== 双臂关节配置 =====
JOINT_CFG_PATH_LEFT = "airexo/configs/joint/left/robot.yaml"
JOINT_CFG_PATH_RIGHT = "airexo/configs/joint/right/robot.yaml"

print("✓ 配置加载成功!")

✓ 配置加载成功!


## 3. 加载数据

In [16]:

class DualArmRobotData:
    """双臂机器人数据加载器"""
    
    def __init__(self, lowdim_h5_path, calib_raw, 
                 joint_cfg_left, joint_cfg_right, 
                 urdf_left, urdf_right):
        self.lowdim_h5_path = lowdim_h5_path
        self.calib_raw = calib_raw
        
        # 左臂配置
        self.urdf_file_left = urdf_left
        self.joint_cfgs_left = OmegaConf.load(joint_cfg_left)
        
        # 右臂配置
        self.urdf_file_right = urdf_right
        self.joint_cfgs_right = OmegaConf.load(joint_cfg_right)
        
        # 加载数据
        self._load_lowdim_data()
        self._process_calibration()
        
    def _load_lowdim_data(self):
        """加载 HDF5 lowdim 数据"""
        print(f"\n📂 加载双臂 lowdim 数据: {self.lowdim_h5_path}")
        
        with h5py.File(self.lowdim_h5_path, 'r') as f:
            # 读取时间戳
            self.timestamps = f['timestamp'][:]
            
            # 从标定数据中识别左右臂相机序列号
            self.cam_serial_left = self.calib_raw['camera_serial_inhand_left']
            self.cam_serial_right = self.calib_raw['camera_serial_inhand_right']
            self.cam_serial_global = self.calib_raw['camera_serials_global'][0]
            
            print(f"\n相机配置:")
            print(f"  - 左臂相机: {self.cam_serial_left}")
            print(f"  - 右臂相机: {self.cam_serial_right}")
            print(f"  - 全局相机: {self.cam_serial_global}")
            
        print(f"\n✓ 加载了 {len(self.timestamps)} 帧数据")
        
    # def _process_calibration(self):
    #     """处理双臂标定数据"""
    #     print(f"\n🎯 处理双臂标定数据...")
        
    #     # 提取内参和外参
    #     intrinsics = self.calib_raw['intrinsics']
    #     extrinsics = self.calib_raw['extrinsics']
        
    #     # 全局相机（用于渲染）
    #     self.intrinsic_global = intrinsics[self.cam_serial_global]
    #     self.extrinsic_global = extrinsics[self.cam_serial_global]
        
    #     # 左臂相机外参（相机系 → 世界系）
    #     self.extrinsic_left = extrinsics[self.cam_serial_left]
    #     self.intrinsic_left = intrinsics[self.cam_serial_left]
        
    #     # 右臂相机外参
    #     self.extrinsic_right = extrinsics[self.cam_serial_right]
    #     self.intrinsic_right = intrinsics[self.cam_serial_right]
        
    #     # 计算世界系 → 相机系变换（用于渲染）
    #     self.world_to_cam = np.linalg.inv(self.extrinsic_global).astype(np.float32)
        
    #     print(f"✓ 标定数据处理完成")

    def _process_calibration(self):
        """处理双臂标定数据 - 使用 generate_calib 逻辑"""
        print(f"\n🎯 处理双臂标定数据...")
        
        # ========== 1. 内参（不变） ==========
        intrinsics = self.calib_raw['intrinsics']
        self.intrinsic_global = intrinsics[self.cam_serial_global]
        self.intrinsic_left = intrinsics[self.cam_serial_left]
        self.intrinsic_right = intrinsics[self.cam_serial_right]
        
        # ========== 2. 计算 camera_to_robot (类似 generate_calib.py) ==========
        # 常量：TCP 到手眼相机的固定变换（从你的机器人参数获取）
        T_CAM_TO_TCP_RAW = np.array([
            [0, -1, 0, 0],
            [1, 0, 0, 0.077],
            [0, 0, 1, 0.2665],
            [0, 0, 0, 1]
        ], dtype=np.float32)
        T_TCP_TO_CAM = np.linalg.inv(T_CAM_TO_TCP_RAW)
        
        # 左臂：Base -> TCP -> Cam -> Marker -> Global
        robot_left = self.calib_raw['robot_left']
        T_base_tcp_left = self._pose7d_to_matrix(robot_left['tcp_pose'])
        
        extrinsics = self.calib_raw['extrinsics']
        T_cam_to_marker_left = extrinsics[self.cam_serial_left]
        
        # 标定板在左臂底座系下的位置
        T_base_marker_left = T_base_tcp_left @ T_TCP_TO_CAM @ T_cam_to_marker_left
        
        # 全局相机到左臂底座的变换
        T_global_to_marker = extrinsics[self.cam_serial_global]
        camera_to_robot_left = T_base_marker_left @ np.linalg.inv(T_global_to_marker)
        
        # 右臂（同理）
        robot_right = self.calib_raw['robot_right']
        T_base_tcp_right = self._pose7d_to_matrix(robot_right['tcp_pose'])
        T_cam_to_marker_right = extrinsics[self.cam_serial_right]
        T_base_marker_right = T_base_tcp_right @ T_TCP_TO_CAM @ T_cam_to_marker_right
        camera_to_robot_right = T_base_marker_right @ np.linalg.inv(T_global_to_marker)
        
        # ========== 3. 存储变换（用于渲染） ==========
        # 机器人底座系 → 全局相机系
        self.robot_to_cam_left = np.linalg.inv(camera_to_robot_left).astype(np.float32)
        self.robot_to_cam_right = np.linalg.inv(camera_to_robot_right).astype(np.float32)
        
        # 全局相机系 → 世界系（这里世界系选择全局相机系本身）
        self.world_to_cam = np.eye(4, dtype=np.float32)  # 恒等变换
        
        print(f"✓ 左臂 Base → Global Cam:")
        print(f"  Z 高度: {camera_to_robot_left[2, 3]:.3f} m")
        print(f"✓ 右臂 Base → Global Cam:")
        print(f"  Z 高度: {camera_to_robot_right[2, 3]:.3f} m")

    def _pose7d_to_matrix(self, pose_7d):
        """7D 位姿转 4x4 矩阵 (兼容四元数格式)"""
        from scipy.spatial.transform import Rotation
        t = pose_7d[:3]
        quat = pose_7d[3:]  # [x, y, z, w] 或 [w, x, y, z]
        
        # 检测格式：如果第一个元素接近1，则为 [w, x, y, z]
        if np.abs(quat[0]) > 0.9:
            quat = [quat[1], quat[2], quat[3], quat[0]]  # 转为 scipy 格式
        
        R_mat = Rotation.from_quat(quat).as_matrix()
        
        T = np.eye(4, dtype=np.float32)
        T[:3, :3] = R_mat
        T[:3, 3] = t
        return T
        
    def get_robot_state_at_frame(self, frame_idx):
        """获取指定帧的双臂状态"""
        robot_left = self.calib_raw.get('robot_left', None)
        robot_right = self.calib_raw.get('robot_right', None)
        
        # 注意：这里假设标定文件中的 robot_left/right 是静态的
        # 如果你的 lowdim.h5 中有每帧的关节角度，需要从那里读取
        
        left_joint = None
        right_joint = None
        
        if robot_left is not None:
            joint_pos_left = robot_left['joint_pos']  # (7,)
            # 夹爪状态需要从其他地方获取，这里暂用 0
            ee_state_left = 0.0
            left_joint = np.concatenate([joint_pos_left, [ee_state_left]])
        
        if robot_right is not None:
            joint_pos_right = robot_right['joint_pos']  # (7,)
            ee_state_right = 0.0
            right_joint = np.concatenate([joint_pos_right, [ee_state_right]])
            
        return left_joint, right_joint


# 创建双臂数据加载器
robot_data = DualArmRobotData(
    lowdim_h5_path=LOWDIM_H5_PATH,
    calib_raw=CALIB_RAW,
    joint_cfg_left=JOINT_CFG_PATH_LEFT,
    joint_cfg_right=JOINT_CFG_PATH_RIGHT,
    urdf_left=URDF_FILE_LEFT,
    urdf_right=URDF_FILE_RIGHT
)



📂 加载双臂 lowdim 数据: /data/haoxiang/data/FLIPPING_v3/train/scene_0001/lowdim/lowdim.h5

相机配置:
  - 左臂相机: 104122064161
  - 右臂相机: 104122061330
  - 全局相机: 105422061350

✓ 加载了 13036 帧数据

🎯 处理双臂标定数据...
✓ 左臂 Base → Global Cam:
  Z 高度: 0.013 m
✓ 右臂 Base → Global Cam:
  Z 高度: -0.080 m


## 4. 正向运动学重建

In [22]:

# def reconstruct_dual_arm_at_frame(robot_data, frame_idx):
#     """
#     在指定帧重建双臂机器人 3D 模型
    
#     返回:
#         meshes_left: 左臂网格列表
#         meshes_right: 右臂网格列表
#     """
#     print(f"\n🤖 重建双臂 - 帧 {frame_idx} (时间戳: {robot_data.timestamps[frame_idx]})")
    
#     # 获取双臂关节状态
#     left_joint, right_joint = robot_data.get_robot_state_at_frame(frame_idx)
    
#     meshes_left = []
#     meshes_right = []
    
#     # ========== 左臂重建 ==========
#     if left_joint is not None:
#         print(f"\n🔧 左臂关节状态: {left_joint[:7]}")
        
#         transforms_left, visuals_left = forward_kinematic_single(
#             joint=left_joint,
#             joint_cfgs=robot_data.joint_cfgs_left,
#             is_rad=True,
#             urdf_file=robot_data.urdf_file_left,
#             with_visuals_map=True
#         )
        
#         urdf_dir_left = os.path.dirname(robot_data.urdf_file_left)
        
#         for link_name, transform in transforms_left.items():
#             visuals = visuals_left.get(link_name, [])
            
#             for visual in visuals:
#                 if visual.geom_param is None:
#                     continue
                
#                 mesh_file = visual.geom_param
#                 if isinstance(mesh_file, (list, tuple)):
#                     mesh_file = mesh_file[0]
                
#                 mesh_path = os.path.join(urdf_dir_left, mesh_file)
#                 if not os.path.exists(mesh_path):
#                     continue
                
#                 # # 变换到世界坐标系
#                 # tf = ROBOT_PREDEFINED_TRANSFORMATION @ \
#                 #      transform.matrix() @ \
#                 #      visual.offset.matrix()

#                 # ✅ 修改这里：左臂变换链
#                 # Base 系 → Link 系 → Mesh 本地系 → 全局相机系
#                 tf = robot_data.robot_to_cam_left @ \
#                     transform.matrix() @ \
#                     visual.offset.matrix()
                
#                 mesh = o3d.io.read_triangle_mesh(mesh_path)
#                 mesh.transform(tf)
#                 mesh.compute_vertex_normals()
#                 mesh.paint_uniform_color([0.3, 0.6, 0.9])  # 蓝色 - 左臂
                
#                 meshes_left.append({
#                     'arm': 'left',
#                     'link_name': link_name,
#                     'mesh_file': mesh_file,
#                     'mesh': mesh,
#                     'transform': tf
#                 })
        
#         print(f"  ✓ 左臂: {len(meshes_left)} 个网格")
    
#     # ========== 右臂重建 ==========
#     if right_joint is not None:
#         print(f"\n🔧 右臂关节状态: {right_joint[:7]}")
        
#         transforms_right, visuals_right = forward_kinematic_single(
#             joint=right_joint,
#             joint_cfgs=robot_data.joint_cfgs_right,
#             is_rad=True,
#             urdf_file=robot_data.urdf_file_right,
#             with_visuals_map=True
#         )
        
#         urdf_dir_right = os.path.dirname(robot_data.urdf_file_right)
        
#         for link_name, transform in transforms_right.items():
#             visuals = visuals_right.get(link_name, [])
            
#             for visual in visuals:
#                 if visual.geom_param is None:
#                     continue
                
#                 mesh_file = visual.geom_param
#                 if isinstance(mesh_file, (list, tuple)):
#                     mesh_file = mesh_file[0]
                
#                 mesh_path = os.path.join(urdf_dir_right, mesh_file)
#                 if not os.path.exists(mesh_path):
#                     continue
                
#                 # tf = ROBOT_PREDEFINED_TRANSFORMATION @ \
#                 #      transform.matrix() @ \
#                 #      visual.offset.matrix()

#                 tf = robot_data.robot_to_cam_right @ \
#                     transform.matrix() @ \
#                     visual.offset.matrix()
                
#                 mesh = o3d.io.read_triangle_mesh(mesh_path)
#                 mesh.transform(tf)
#                 mesh.compute_vertex_normals()
#                 mesh.paint_uniform_color([0.9, 0.3, 0.3])  # 红色 - 右臂
                
#                 meshes_right.append({
#                     'arm': 'right',
#                     'link_name': link_name,
#                     'mesh_file': mesh_file,
#                     'mesh': mesh,
#                     'transform': tf
#                 })
        
#         print(f"  ✓ 右臂: {len(meshes_right)} 个网格")
    
#     print(f"\n✓ 双臂重建完成: 左臂 {len(meshes_left)} + 右臂 {len(meshes_right)} = {len(meshes_left) + len(meshes_right)} 个网格")
    
#     return meshes_left, meshes_right

def reconstruct_dual_arm_at_frame(robot_data, frame_idx):
    """
    在指定帧重建双臂机器人 3D 模型（保持在各自底座系）
    """
    print(f"\n🤖 重建双臂 - 帧 {frame_idx}")
    
    left_joint, right_joint = robot_data.get_robot_state_at_frame(frame_idx)
    
    meshes_left = []
    meshes_right = []
    
    # ========== 左臂重建 ==========
    if left_joint is not None:
        print(f"\n🔧 左臂关节状态: {left_joint[:7]}")
        
        transforms_left, visuals_left = forward_kinematic_single(
            joint=left_joint,
            joint_cfgs=robot_data.joint_cfgs_left,
            is_rad=True,
            urdf_file=robot_data.urdf_file_left,
            with_visuals_map=True
        )
        
        urdf_dir_left = os.path.dirname(robot_data.urdf_file_left)
        
        for link_name, transform in transforms_left.items():
            visuals = visuals_left.get(link_name, [])
            
            for visual in visuals:
                if visual.geom_param is None:
                    continue
                
                mesh_file = visual.geom_param
                if isinstance(mesh_file, (list, tuple)):
                    mesh_file = mesh_file[0]
                
                mesh_path = os.path.join(urdf_dir_left, mesh_file)
                if not os.path.exists(mesh_path):
                    continue
                
                # ✅ 关键修改：只保持在底座系，不转到世界系
                # tf_base: mesh 在左臂底座系下的位置
                tf_base = transform.matrix() @ visual.offset.matrix()
                
                mesh = o3d.io.read_triangle_mesh(mesh_path)
                mesh.transform(tf_base)  # 只应用到底座系
                mesh.compute_vertex_normals()
                mesh.paint_uniform_color([0.3, 0.6, 0.9])  # 蓝色
                
                meshes_left.append({
                    'arm': 'left',
                    'link_name': link_name,
                    'mesh_file': mesh_file,
                    'mesh': mesh,
                    'transform_base': tf_base  # 保存底座系变换
                })
        
        print(f"  ✓ 左臂: {len(meshes_left)} 个网格")
    
    # ========== 右臂重建（同理） ==========
    if right_joint is not None:
        print(f"\n🔧 右臂关节状态: {right_joint[:7]}")
        
        transforms_right, visuals_right = forward_kinematic_single(
            joint=right_joint,
            joint_cfgs=robot_data.joint_cfgs_right,
            is_rad=True,
            urdf_file=robot_data.urdf_file_right,
            with_visuals_map=True
        )
        
        urdf_dir_right = os.path.dirname(robot_data.urdf_file_right)
        
        for link_name, transform in transforms_right.items():
            visuals = visuals_right.get(link_name, [])
            
            for visual in visuals:
                if visual.geom_param is None:
                    continue
                
                mesh_file = visual.geom_param
                if isinstance(mesh_file, (list, tuple)):
                    mesh_file = mesh_file[0]
                
                mesh_path = os.path.join(urdf_dir_right, mesh_file)
                if not os.path.exists(mesh_path):
                    continue
                
                tf_base = transform.matrix() @ visual.offset.matrix()
                
                mesh = o3d.io.read_triangle_mesh(mesh_path)
                mesh.transform(tf_base)
                mesh.compute_vertex_normals()
                mesh.paint_uniform_color([0.9, 0.3, 0.3])  # 红色
                
                meshes_right.append({
                    'arm': 'right',
                    'link_name': link_name,
                    'mesh_file': mesh_file,
                    'mesh': mesh,
                    'transform_base': tf_base
                })
        
        print(f"  ✓ 右臂: {len(meshes_right)} 个网格")
    
    print(f"\n✓ 双臂重建完成")
    
    return meshes_left, meshes_right


# 测试双臂重建
FRAME_IDX = 0
meshes_left, meshes_right = reconstruct_dual_arm_at_frame(robot_data, FRAME_IDX)



🤖 重建双臂 - 帧 0

🔧 左臂关节状态: [ 0.44278383 -1.06347775  1.39281845 -1.55827224 -1.60449004 -0.49928525
  0.33921626]
  ✓ 左臂: 17 个网格

🔧 右臂关节状态: [ 0.38581365 -0.71891367  1.92083669 -1.4884975  -1.99565065 -0.03493906
  0.42234862]
  ✓ 右臂: 17 个网格

✓ 双臂重建完成


## 5. 3D 可视化

In [23]:
import k3d
def visualize_dual_arm_k3d(meshes_left, meshes_right, show_coordinate_frame=True):
    """
    使用 K3D 可视化双臂机器人
    
    蓝色: 左臂
    红色: 右臂
    """
    plot = k3d.plot(
        grid_visible=True,
        grid=[-1, -1, -1, 1, 1, 1],
        background_color=0xffffff,
        menu_visibility=True
    )
    
    # 坐标系
    if show_coordinate_frame:
        axis_length = 0.2
        origin = ROBOT_PREDEFINED_TRANSFORMATION[:3, 3].astype(np.float32)
        rot = ROBOT_PREDEFINED_TRANSFORMATION[:3, :3].astype(np.float32)
        
        x_end = origin + rot[:, 0] * axis_length
        y_end = origin + rot[:, 1] * axis_length
        z_end = origin + rot[:, 2] * axis_length
        
        plot += k3d.line(np.stack([origin, x_end]), color=0xff0000, width=0.01, name="X-axis")
        plot += k3d.line(np.stack([origin, y_end]), color=0x00ff00, width=0.01, name="Y-axis")
        plot += k3d.line(np.stack([origin, z_end]), color=0x0000ff, width=0.01, name="Z-axis")
    
    # 左臂 - 蓝色系
    print("🔵 添加左臂网格...")
    for mesh_data in meshes_left:
        mesh = mesh_data['mesh']
        vertices = np.asarray(mesh.vertices).astype(np.float32)
        indices = np.asarray(mesh.triangles).astype(np.uint32)
        
        k3d_mesh = k3d.mesh(
            vertices, indices,
            color=0x4D96FF,  # 蓝色
            name=f"LEFT_{mesh_data['link_name']}",
            opacity=1.0,
            side='double'
        )
        plot += k3d_mesh
    
    # 右臂 - 红色系
    print("🔴 添加右臂网格...")
    for mesh_data in meshes_right:
        mesh = mesh_data['mesh']
        vertices = np.asarray(mesh.vertices).astype(np.float32)
        indices = np.asarray(mesh.triangles).astype(np.uint32)
        
        k3d_mesh = k3d.mesh(
            vertices, indices,
            color=0xFF4D4D,  # 红色
            name=f"RIGHT_{mesh_data['link_name']}",
            opacity=1.0,
            side='double'
        )
        plot += k3d_mesh
    
    plot.display()
    
    print("\n🎮 K3D 操作指南:")
    print("  - 左键拖拽: 旋转")
    print("  - 右键拖拽: 平移")
    print("  - 滚轮: 缩放")
    print("  - 🔵 蓝色: 左臂")
    print("  - 🔴 红色: 右臂")
    
    return plot


# 可视化双臂
plot_instance = visualize_dual_arm_k3d(meshes_left, meshes_right)


🔵 添加左臂网格...
🔴 添加右臂网格...


Output()


🎮 K3D 操作指南:
  - 左键拖拽: 旋转
  - 右键拖拽: 平移
  - 滚轮: 缩放
  - 🔵 蓝色: 左臂
  - 🔴 红色: 右臂


## 6. 渲染 RGB 图像

In [ ]:

def render_dual_arm_image(robot_data, meshes_left, meshes_right, width=1280, height=720):
    """
    渲染双臂机器人到 RGB 图像
    """
    print(f"\n🎨 渲染双臂 {width}x{height} 图像...")
    
    renderer = o3d.visualization.rendering.OffscreenRenderer(width, height)
    
    material = o3d.visualization.rendering.MaterialRecord()
    material.shader = "defaultLit"
    
    # 合并所有网格
    all_meshes = meshes_left + meshes_right
    
    # 添加到场景（变换到相机坐标系）
    for i, mesh_data in enumerate(all_meshes):
        mesh = mesh_data['mesh']
        mesh_copy = o3d.geometry.TriangleMesh(mesh)
        
        # 变换到相机系
        mesh_copy.transform(O3D_RENDER_TRANSFORMATION @ robot_data.world_to_cam)
        
        renderer.scene.add_geometry(f"mesh_{i}", mesh_copy, material)
    
    # 设置相机
    renderer.scene.camera.set_projection(
        robot_data.intrinsic_global,
        0.01, 100.0,
        float(width), float(height)
    )
    
    # 渲染
    rgb_image = np.asarray(renderer.render_to_image(), dtype=np.uint8)
    depth_image = np.asarray(
        renderer.render_to_depth_image(z_in_view_space=True),
        dtype=np.float32
    )
    
    print(f"✓ 渲染完成: RGB {rgb_image.shape}, Depth {depth_image.shape}")
    
    return rgb_image, depth_image


# 渲染双臂
rgb_image, depth_image = render_dual_arm_image(robot_data, meshes_left, meshes_right)


## 7. 显示渲染结果

In [ ]:

plt.figure(figsize=(15, 5))

# RGB
plt.subplot(1, 2, 1)
plt.imshow(rgb_image)
plt.title(f"双臂机器人 RGB 渲染 - Frame {FRAME_IDX}")
plt.axis('off')

# Depth
plt.subplot(1, 2, 2)
depth_vis = depth_image.copy()
depth_vis[depth_vis == np.inf] = 0
depth_vis = (depth_vis / depth_vis.max() * 255).astype(np.uint8)
plt.imshow(depth_vis, cmap='jet')
plt.title(f"深度图 - Frame {FRAME_IDX}")
plt.colorbar()
plt.axis('off')

plt.tight_layout()
plt.show()


## 机械臂和点云对照查看

In [24]:

import k3d
import cv2
import os

def debug_dual_arm_alignment_k3d(robot_data, frame_idx, 
                                  rgb_path_global, depth_path_global,
                                  subsample_step=4):
    """
    在 K3D 中同时显示（均在全局相机坐标系下）：
    1. 深度图反投影得到的点云（全局相机系）
    2. 双臂 FK 机器人模型经 world_to_cam 映射到全局相机系后的 mesh
    
    参数:
        robot_data: DualArmRobotData 对象
        frame_idx: 帧索引
        rgb_path_global: 全局相机 RGB 图像路径
        depth_path_global: 全局相机深度图路径
        subsample_step: 点云降采样步长（越大越快）
    """
    
    # ========== 0. 路径检查 ==========
    print(f"\n📂 检查文件路径...")
    print(f"  RGB:   {rgb_path_global}")
    print(f"  Depth: {depth_path_global}")
    
    if not os.path.exists(rgb_path_global):
        raise FileNotFoundError(f"❌ RGB 图像不存在: {rgb_path_global}")
    if not os.path.exists(depth_path_global):
        raise FileNotFoundError(f"❌ 深度图不存在: {depth_path_global}")
    
    print("  ✓ 文件存在")
    
    # ========== 1. 点云：深度反投影到全局相机系 ==========
    print("\n☁️ 生成点云（全局相机坐标系）...")
    
    # 加载全局相机图像
    color_img = cv2.imread(rgb_path_global)
    if color_img is None:
        raise ValueError(f"❌ 无法读取 RGB 图像: {rgb_path_global}")
    color_img = cv2.cvtColor(color_img, cv2.COLOR_BGR2RGB)
    
    depth_img = cv2.imread(depth_path_global, cv2.IMREAD_UNCHANGED)
    if depth_img is None:
        raise ValueError(f"❌ 无法读取深度图: {depth_path_global}")
    depth_img = depth_img.astype(np.float32)
    
    print(f"  ✓ RGB 图像: {color_img.shape}")
    print(f"  ✓ 深度图: {depth_img.shape}, 范围: [{depth_img.min():.1f}, {depth_img.max():.1f}]")
    
    # 获取全局相机内参
    K = robot_data.intrinsic_global
    fx, fy = K[0, 0], K[1, 1]
    cx, cy = K[0, 2], K[1, 2]
    print(f"  ✓ 内参: fx={fx:.1f}, fy={fy:.1f}, cx={cx:.1f}, cy={cy:.1f}")
    
    # 降采样生成点云
    v_idx, u_idx = np.indices(depth_img.shape)
    v_idx = v_idx[::subsample_step, ::subsample_step].flatten()
    u_idx = u_idx[::subsample_step, ::subsample_step].flatten()
    z = depth_img[::subsample_step, ::subsample_step].flatten()
    colors = color_img[::subsample_step, ::subsample_step].reshape(-1, 3)
    
    # 深度过滤
    valid_mask = (z > 0) & (z < 2000)  # 0-2000mm
    u_idx = u_idx[valid_mask]
    v_idx = v_idx[valid_mask]
    z = z[valid_mask]
    colors = colors[valid_mask]
    
    print(f"  ✓ 有效点数: {len(z):,} / {len(valid_mask):,} ({100*len(z)/len(valid_mask):.1f}%)")
    
    if len(z) == 0:
        raise ValueError("❌ 没有有效的深度点！请检查深度图单位或过滤范围")
    
    # 深度转米（假设深度单位是毫米）
    scale = 1000.0
    z_m = z / scale
    
    # 反投影到全局相机坐标系
    x_m = (u_idx - cx) * z_m / fx
    y_m = (v_idx - cy) * z_m / fy
    
    points_cam = np.stack([x_m, y_m, z_m], axis=1)
    
    print(f"  ✓ 点云范围: X=[{x_m.min():.2f}, {x_m.max():.2f}], "
          f"Y=[{y_m.min():.2f}, {y_m.max():.2f}], "
          f"Z=[{z_m.min():.2f}, {z_m.max():.2f}] (米)")
    
    # 颜色转 K3D 格式
    r = colors[:, 0].astype(np.uint32)
    g = colors[:, 1].astype(np.uint32)
    b = colors[:, 2].astype(np.uint32)
    colors_int = (r << 16) | (g << 8) | b
    
    # ========== 2. 双臂机器人 FK 并映射到全局相机系 ==========
    print("\n🤖 FK 计算双臂机器人并映射到全局相机系...")
    
    meshes_left, meshes_right = reconstruct_dual_arm_at_frame(robot_data, frame_idx)
    
    # 世界系 → 全局相机系的变换
    world_to_cam = robot_data.world_to_cam
    
    # ========== 3. K3D 可视化 ==========
    print("\n🎨 正在构建 K3D 场景...")
    plot = k3d.plot(
        background_color=0xFFFFFF,
        grid_visible=True,
        camera_auto_fit=True
    )
    
    # A. 点云（全局相机系）
    cloud_plt = k3d.points(
        positions=points_cam.astype(np.float32),
        colors=colors_int.astype(np.uint32),
        point_size=0.004,
        shader='flat',
        name="📷 Point Cloud (Global Camera)"
    )
    plot += cloud_plt
    
    # B. 左臂机器人（蓝色）
    print("🔵 添加左臂到场景...")
    for item in meshes_left:
        mesh_o3d = item['mesh']
        verts = np.asarray(mesh_o3d.vertices)
        faces = np.asarray(mesh_o3d.triangles)
        
        # 变换到全局相机系
        verts_h = np.hstack([verts, np.ones((len(verts), 1))])
        verts_cam = (world_to_cam @ verts_h.T).T[:, :3]
        
        mesh_plt = k3d.mesh(
            vertices=verts_cam.astype(np.float32),
            indices=faces.astype(np.uint32),
            color=0x4D96FF,  # 蓝色
            opacity=0.8,
            name=f"🔵 LEFT_{item['link_name']}"
        )
        plot += mesh_plt
    
    # C. 右臂机器人（红色）
    print("🔴 添加右臂到场景...")
    for item in meshes_right:
        mesh_o3d = item['mesh']
        verts = np.asarray(mesh_o3d.vertices)
        faces = np.asarray(mesh_o3d.triangles)
        
        # 变换到全局相机系
        verts_h = np.hstack([verts, np.ones((len(verts), 1))])
        verts_cam = (world_to_cam @ verts_h.T).T[:, :3]
        
        mesh_plt = k3d.mesh(
            vertices=verts_cam.astype(np.float32),
            indices=faces.astype(np.uint32),
            color=0xFF4D4D,  # 红色
            opacity=0.8,
            name=f"🔴 RIGHT_{item['link_name']}"
        )
        plot += mesh_plt

    # transform_matrix = np.linalg.inv(world_to_cam) 

    # # B. 左臂机器人（蓝色）
    # print("🔵 添加左臂到场景...")
    # for item in meshes_left:
    #     mesh_o3d = item['mesh']
    #     verts = np.asarray(mesh_o3d.vertices)
    #     faces = np.asarray(mesh_o3d.triangles)
        
    #     # 变换到全局相机系
    #     verts_h = np.hstack([verts, np.ones((len(verts), 1))])
        
    #     # 【修改处】使用求逆后的 transform_matrix 进行运算
    #     # 现在的运算逻辑是: P_new = (M^-1 @ P_old^T)^T
    #     verts_cam = (transform_matrix @ verts_h.T).T[:, :3]
        
    #     mesh_plt = k3d.mesh(
    #         vertices=verts_cam.astype(np.float32),
    #         indices=faces.astype(np.uint32),
    #         color=0x4D96FF,  # 蓝色
    #         opacity=0.8,
    #         name=f"🔵 LEFT_{item['link_name']}"
    #     )
    #     plot += mesh_plt

    # # C. 右臂机器人（红色）
    # print("🔴 添加右臂到场景...")
    # for item in meshes_right:
    #     mesh_o3d = item['mesh']
    #     verts = np.asarray(mesh_o3d.vertices)
    #     faces = np.asarray(mesh_o3d.triangles)
        
    #     # 变换到全局相机系
    #     verts_h = np.hstack([verts, np.ones((len(verts), 1))])
        
    #     # 【修改处】同样使用求逆后的 transform_matrix
    #     verts_cam = (transform_matrix @ verts_h.T).T[:, :3]
        
    #     mesh_plt = k3d.mesh(
    #         vertices=verts_cam.astype(np.float32),
    #         indices=faces.astype(np.uint32),
    #         color=0xFF4D4D,  # 红色
    #         opacity=0.8,
    #         name=f"🔴 RIGHT_{item['link_name']}"
    #     )
    #     plot += mesh_plt
    
    # D. 坐标轴：全局相机系原点 (X=红, Y=绿, Z=蓝)
    axis_len = 0.2
    plot += k3d.line([[0,0,0], [axis_len,0,0]], color=0xff0000, width=0.015, name="X-axis")
    plot += k3d.line([[0,0,0], [0,axis_len,0]], color=0x00ff00, width=0.015, name="Y-axis")
    plot += k3d.line([[0,0,0], [0,0,axis_len]], color=0x0000ff, width=0.015, name="Z-axis")
    plot += k3d.points([[0,0,0]], color=0xffff00, point_size=0.03, name="Camera Origin")
    
    plot.display()
    


# ==============================
# 辅助函数：自动查找时间戳最近的图像
# ==============================
def find_closest_image(scene_path, camera_serial, target_timestamp, image_type='color'):
    """
    在指定相机目录下查找最接近目标时间戳的图像
    
    参数:
        scene_path: 场景根目录
        camera_serial: 相机序列号
        target_timestamp: 目标时间戳 (整数)
        image_type: 'color' 或 'depth'
    
    返回:
        图像完整路径
    """
    img_dir = os.path.join(scene_path, f"cam_{camera_serial}", image_type)
    
    if not os.path.exists(img_dir):
        raise FileNotFoundError(f"相机目录不存在: {img_dir}")
    
    # 获取所有图像文件
    files = [f for f in os.listdir(img_dir) if f.endswith('.png')]
    
    if len(files) == 0:
        raise FileNotFoundError(f"目录中没有图像: {img_dir}")
    
    # 提取时间戳
    timestamps = [int(os.path.splitext(f)[0]) for f in files]
    
    # 找到最接近的时间戳
    closest_idx = np.argmin(np.abs(np.array(timestamps) - target_timestamp))
    closest_file = files[closest_idx]
    
    print(f"  📷 {camera_serial}/{image_type}: {closest_file} "
          f"(Δt = {timestamps[closest_idx] - target_timestamp} ms)")
    
    return os.path.join(img_dir, closest_file)


# ==============================
# 运行双臂对齐调试（自动查找图像）
# ==============================
SCENE_PATH = "/data/haoxiang/data/airexo2/task_0013/train/scene_0001"
target_idx = 0
target_timestamp = robot_data.timestamps[target_idx]

print(f"\n🎯 目标帧: {target_idx}, 时间戳: {target_timestamp}")
print("\n🔍 自动查找最接近的图像...")


# 自动查找全局相机的 RGB 和深度图
rgb_path_global = find_closest_image(
    SCENE_PATH, 
    robot_data.cam_serial_global, 
    target_timestamp, 
    'color'
)

depth_path_global = find_closest_image(
    SCENE_PATH, 
    robot_data.cam_serial_global, 
    target_timestamp, 
    'depth'
)

# 运行可视化
debug_dual_arm_alignment_k3d(
    robot_data, 
    target_idx, 
    rgb_path_global, 
    depth_path_global,
    subsample_step=4  # 调整这个参数控制点云密度
)
    




🎯 目标帧: 0, 时间戳: 1767593840232

🔍 自动查找最接近的图像...
  📷 105422061350/color: 1737546151366.png (Δt = -30047688866 ms)
  📷 105422061350/depth: 1737546151366.png (Δt = -30047688866 ms)

📂 检查文件路径...
  RGB:   /data/haoxiang/data/airexo2/task_0013/train/scene_0001/cam_105422061350/color/1737546151366.png
  Depth: /data/haoxiang/data/airexo2/task_0013/train/scene_0001/cam_105422061350/depth/1737546151366.png
  ✓ 文件存在

☁️ 生成点云（全局相机坐标系）...
  ✓ RGB 图像: (720, 1280, 3)
  ✓ 深度图: (720, 1280), 范围: [0.0, 2539.0]
  ✓ 内参: fx=912.4, fy=911.5, cx=633.4, cy=364.2
  ✓ 有效点数: 42,266 / 57,600 (73.4%)
  ✓ 点云范围: X=[-1.11, 1.06], Y=[-0.76, 0.76], Z=[0.69, 2.00] (米)

🤖 FK 计算双臂机器人并映射到全局相机系...

🤖 重建双臂 - 帧 0

🔧 左臂关节状态: [ 0.44278383 -1.06347775  1.39281845 -1.55827224 -1.60449004 -0.49928525
  0.33921626]
  ✓ 左臂: 17 个网格

🔧 右臂关节状态: [ 0.38581365 -0.71891367  1.92083669 -1.4884975  -1.99565065 -0.03493906
  0.42234862]
  ✓ 右臂: 17 个网格

✓ 双臂重建完成

🎨 正在构建 K3D 场景...
🔵 添加左臂到场景...
🔴 添加右臂到场景...


Output()

## 8. 使用 RobotRenderer 类 (高级用法)

In [ ]:
# 使用 AirExo-2 的 RobotRenderer 类进行渲染
# 注意:这需要双臂配置,但我们可以适配为单臂

print("\n🎬 使用 RobotRenderer 类渲染...")

# 创建一个虚拟的右臂配置 (全零)
class DummyJointConfig:
    def __init__(self):
        self.num_joints = 8
        self.num_robot_joints = 7

renderer = RobotRenderer(
    # left_joint_cfgs=DummyJointConfig(), # 虚拟左臂
    # right_joint_cfgs=robot_data.joint_cfgs,  
    left_joint_cfgs = robot_data.joint_cfgs, 
    right_joint_cfgs = DummyJointConfig(),
    cam_to_base=robot_data.cam_to_base,
    intrinsic=robot_data.intrinsic,
    width=1280,
    height=720,
    near_plane=0.01,
    far_plane=100.0,
    # urdf_file=os.path.join(AIREXO_PATH, "airexo/urdf_models/robot/robot.urdf")
    urdf_file="airexo/urdf_models/robot/robot.urdf"
)

# 更新关节状态
# right_joint = robot_data.get_joint_at_timestamp(FRAME_IDX)
# left_joint = np.zeros(8)  # 虚拟左臂
right_joint = np.zeros(8)
left_joint = robot_data.get_joint_at_timestamp(FRAME_IDX)


renderer.update_joints(left_joint, right_joint)

# 渲染
rendered_image = renderer.render_image()
rendered_depth = renderer.render_depth()

# 显示
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.imshow(rendered_image)
plt.title("RobotRenderer - RGB")
plt.axis('off')

plt.subplot(1, 2, 2)
depth_vis = (rendered_depth / rendered_depth.max() * 255).astype(np.uint8)
plt.imshow(depth_vis, cmap='jet')
plt.title("RobotRenderer - Depth")
plt.axis('off')

plt.tight_layout()
plt.show()

print("\n✓ RobotRenderer 渲染完成!")

## 9. 批量处理多帧

In [ ]:
def batch_render_frames(robot_data, frame_indices, output_dir="./batch_output"):
    """
    批量渲染多帧
    
    参数:
        robot_data: 数据加载器
        frame_indices: 要渲染的帧索引列表
        output_dir: 输出目录
    """
    os.makedirs(output_dir, exist_ok=True)
    print(f"\n📹 批量渲染 {len(frame_indices)} 帧...")
    
    for i, frame_idx in enumerate(frame_indices):
        print(f"\n[{i+1}/{len(frame_indices)}] 处理帧 {frame_idx}")
        
        # 重建
        _, _, meshes = reconstruct_robot_at_frame(robot_data, frame_idx)
        
        # 渲染
        rgb, depth = render_robot_image(robot_data, meshes)
        
        # 保存
        output_path = os.path.join(output_dir, f"frame_{frame_idx:06d}.png")
        Image.fromarray(rgb).save(output_path)
        print(f"  ✓ 已保存: {output_path}")
    
    print(f"\n✓ 批量渲染完成! 输出目录: {output_dir}")


# 示例: 渲染第 0, 100, 200, 300, 400 帧
batch_render_frames(
    robot_data,
    frame_indices=[0, 100, 200, 300, 400],
    output_dir="./reconstruction_output"
)

## 10. 导出完整 3D 模型

In [ ]:
def export_3d_model(meshes, output_path="robot_model.ply"):
    """
    导出合并后的 3D 模型
    
    参数:
        meshes: 网格列表
        output_path: 输出文件路径
    """
    print(f"\n💾 导出 3D 模型...")
    
    # 合并所有网格
    combined_mesh = o3d.geometry.TriangleMesh()
    for mesh_data in meshes:
        combined_mesh += mesh_data['mesh']
    
    # 重新计算法线
    combined_mesh.compute_vertex_normals()
    
    # 保存
    o3d.io.write_triangle_mesh(output_path, combined_mesh)
    
    print(f"✓ 3D 模型已保存: {output_path}")
    print(f"  - 顶点数: {len(combined_mesh.vertices)}")
    print(f"  - 三角形数: {len(combined_mesh.triangles)}")


# 导出第 100 帧的 3D 模型
export_3d_model(meshes, output_path=f"robot_frame_{FRAME_IDX}.ply")

## 总结

本 Notebook 展示了如何使用 AirExo-2 的原生函数从 lowdim 数据重建单臂机器人的 3D 模型:

1. ✅ **数据加载**: 从 HDF5 文件读取关节角度和标定数据
2. ✅ **正向运动学**: 使用 `forward_kinematic_single` 计算 link 变换
3. ✅ **3D 重建**: 加载 URDF 网格并应用变换
4. ✅ **可视化**: 使用 Open3D 进行交互式 3D 可视化
5. ✅ **渲染**: 使用离屏渲染器生成 RGB 和深度图
6. ✅ **批量处理**: 支持多帧批量渲染
7. ✅ **模型导出**: 导出 PLY 格式的 3D 模型

### 关键技术点

- **标定**: pose_in_link 为 7 维 (xyz+四元数)，表示相机在机械臂 base 下的位姿；cam_to_base 为相机系→底座系，base_to_cam 为其逆。
- **渲染/对齐**: mesh 在底座系，经 base_to_cam 映射到相机系后与点云（相机系）一致；渲染时变换链为 base_to_cam → O3D_RENDER_TRANSFORMATION。
- **关节数据**: 7个机器人关节 (弧度) + 1个夹爪宽度
- **URDF 解析**: 使用 `kinpy` 库进行正向运动学
- **渲染管线**: Open3D 离屏渲染 + 相机内参投影

### 下一步

- 🎥 制作动画序列
- 🔄 与真实 RGB 图像叠加对比
- 📊 分析 TCP 位姿误差
- 🎮 集成到数据可视化工具